In [61]:
from omero.gateway import BlitzGateway
from data_preperation import *

In [62]:
conn = BlitzGateway("Asterix", "abc123",group='microscope_1', host="localhost", port=6064, secure=True)
conn.connect()

True

In [32]:
user = conn.getUser()
print("Current user:")
print("   ID:", user.getId())
print("   Username:", user.getName())
print("   Full Name:", user.getFullName())

# Check if you are an Administrator
print("   Is Admin:", conn.isAdmin())
if not conn.isFullAdmin():
    # If 'Restricted Administrator' show privileges
    print(conn.getCurrentAdminPrivileges())

print("Member of:")
for g in conn.getGroupsMemberOf():
    print("   ID:", g.getName(), " Name:", g.getId())
group = conn.getGroupFromContext()
print("Current group: ", group.getName())

# List the group owners and other members
owners, members = group.groupSummary()
print("   Group owners:")
for o in owners:
    print("     ID: %s %s Name: %s" % (
        o.getId(), o.getOmeName(), o.getFullName()))
print("   Group members:")
for m in members:
    print("     ID: %s %s Name: %s" % (
        m.getId(), m.getOmeName(), m.getFullName()))

print("Owner of:")
for g in conn.listOwnedGroups():
    print("   ID: ", g.getName(), " Name:", g.getId())

# Added in OMERO 5.0
print("Admins:")
for exp in conn.getAdministrators():
    print("   ID: %s %s Name: %s" % (
        exp.getId(), exp.getOmeName(), exp.getFullName()))

# The 'context' of our current session
ctx = conn.getEventContext()
# print(ctx)     # for more info

Current user:
   ID: 3
   Username: Asterix
   Full Name: Asterix CoreFacilityStaff
   Is Admin: False
[]
Member of:
   ID: core_facility_staff  Name: 4
   ID: microscope_1  Name: 5
   ID: microscope_2  Name: 6
Current group:  core_facility_staff
   Group owners:
     ID: 2 Abraracurcix Name: Abraracurcix CoreFacilityManager
   Group members:
     ID: 5 Panoramix Name: Panoramix CoreFacilityStaff
     ID: 3 Asterix Name: Asterix CoreFacilityStaff
     ID: 4 Obelix Name: Obelix CoreFacilityStaff
Owner of:
   ID:  microscope_1  Name: 5
   ID:  microscope_2  Name: 6
Admins:
   ID: 0 root Name: root root


In [4]:
for group in conn.listOwnedGroups():
    owners, members = group.groupSummary()
    print("   Group owners:")
    for o in owners:
        print("     ID: %s %s Name: %s" % (
            o.getId(), o.getOmeName(), o.getFullName()))
    print("   Group members:")
    for m in members:
        print("     ID: %s %s Name: %s" % (
            m.getId(), m.getOmeName(), m.getFullName()))

   Group owners:
     ID: 3 Asterix Name: Asterix CoreFacilityStaff
     ID: 4 Obelix Name: Obelix CoreFacilityStaff
   Group members:
     ID: 5 Panoramix Name: Panoramix CoreFacilityStaff
     ID: 2 Abraracurcix Name: Abraracurcix CoreFacilityManager
   Group owners:
     ID: 3 Asterix Name: Asterix CoreFacilityStaff
     ID: 5 Panoramix Name: Panoramix CoreFacilityStaff
   Group members:
     ID: 4 Obelix Name: Obelix CoreFacilityStaff
     ID: 2 Abraracurcix Name: Abraracurcix CoreFacilityManager


In [52]:
roi_ids = [r.getId().getValue() for r in result.rois]

In [53]:
roi_ids

[7, 10, 13, 16, 19, 22]

In [63]:
roi_service = conn.getRoiService()
result = roi_service.findByImage(int(3), None, conn.SERVICE_OPTS)
shapes_rectangle, shapes_line = get_rois_omero(result)
df_lines_omero = get_info_roi_lines(shapes_line)

AttributeError: 'NoneType' object has no attribute 'getValue'

In [67]:
for i in conn.listOwnedGroups():
    print(f'------------------------------{i.getName()}----------------------------------')
    conn.SERVICE_OPTS.setOmeroGroup(i.getId())
    for project in conn.getObjects("Project"):
        print(project.getName(), project.getId())

------------------------------microscope_1----------------------------------
Field Illumination 60x 1
Field Illumination 10x 2
PSF beads 60x 3
------------------------------microscope_2----------------------------------
Field Illumination 40x 4
Field Illumination 20x 5
PSF beads 40x 6


In [59]:

shapes_line = {}
shapes_rectangle = {}
for roi in result.rois:
    for s in roi.copyShapes():
        shape = {}
        shape["id"] = s.getId().getValue()
        if s.getTextValue():
            shape["textValue"] = s.getTextValue().getValue()
        if s.__class__.__name__ == "RectangleI":
            shape["type"] = "Rectangle"
            shape["x"] = s.getX().getValue()
            shape["y"] = s.getY().getValue()
            shape["w"] = s.getWidth().getValue()
            shape["h"] = s.getHeight().getValue()
            shapes_rectangle[s.getId().getValue()] = shape
        elif s.__class__.__name__ == "LineI":
            shape["type"] = "Line"
            shape["x1"] = s.getX1().getValue()
            shape["x2"] = s.getX2().getValue()
            shape["y1"] = s.getY1().getValue()
            shape["y2"] = s.getY2().getValue()
            shapes_line[s.getId().getValue()] = shape
        elif s.__class__.__name__ == "PolygonI":
            continue


In [28]:
g = conn.listProjects()

In [47]:
for project in conn.getObjects("Project", opts={'group': 4}):
    print(project.getName(), project.getId())

Field Illumination 60x 1
Field Illumination 10x 2
Field Illumination 40x 4
Field Illumination 20x 5
PSF beads 60x 3
PSF beads 40x 6


In [ ]:
def get_data_ids(conn):
    for i in conn.listOwnedGroups():
        conn.SERVICE_OPTS.setOmeroGroup(i.getId())
        for project in conn.getObjects("Project"):
            print(project.getName(), project.getId())

In [68]:
conn.SERVICE_OPTS.setOmeroGroup(-1)
for image in conn.getObjects("Image"):
    print(image.getName(), image.getId())

FI_60x_DAPI_2021-02-01 1
FI_60x_FITC_2021-02-01 2
FI_60x_TRITC_CY5_2021-02-01 3
FI_60x_DAPI_2021-03-01 4
FI_60x_FITC_2021-03-01 5
FI_60x_TRITC_CY5_2021-03-01 6
FI_10x_DAPI_2021-03-04 7
FI_10x_FITC_2021-03-04 8
FI_10x_TRITC_CY5_2021-03-04 9
FI_10x_DAPI_2021-05-04 10
FI_10x_FITC_2021-05-04 11
FI_10x_TRITC_CY5_2021-05-04 12
FI_10x_DAPI_2021-07-04 13
FI_10x_FITC_2021-07-04 14
FI_10x_TRITC_CY5_2021-07-04 15
FI_10x_DAPI_2021-09-04 16
FI_10x_FITC_2021-09-04 17
FI_10x_TRITC_CY5_2021-09-04 18
FI_60x_DAPI_2021-02-01 21
FI_60x_FITC_2021-02-01 22
FI_60x_TRITC_CY5_2021-02-01 23
FI_60x_DAPI_2021-03-01 24
FI_60x_FITC_2021-03-01 25
FI_60x_TRITC_CY5_2021-03-01 26
FI_10x_DAPI_2021-03-04 27
FI_10x_FITC_2021-03-04 28
FI_10x_TRITC_CY5_2021-03-04 29
FI_10x_DAPI_2021-05-04 30
FI_10x_FITC_2021-05-04 31
FI_10x_TRITC_CY5_2021-05-04 32
FI_10x_DAPI_2021-07-04 33
FI_10x_FITC_2021-07-04 34
FI_10x_TRITC_CY5_2021-07-04 35
FI_10x_DAPI_2021-09-04 36
FI_10x_FITC_2021-09-04 37
FI_10x_TRITC_CY5_2021-09-04 38
PSF-beads_60x

In [5]:
dataset_4 = conn.getObject("Dataset", 52)
dataset_6 = conn.getObject("Dataset", 6)


In [6]:
dataset_4.getAnnotation()

<MapAnnotationWrapper id=54>

In [34]:
dataset_4.getAnnotationCounts()

{'TagAnnotation': 0,
 'FileAnnotation': 0,
 'CommentAnnotation': 0,
 'LongAnnotation': 0,
 'MapAnnotation': 2,
 'OtherAnnotation': 0}

In [35]:
an= conn.getObject("MapAnnotation", 6)


In [36]:
ann = dataset_4.getAnnotation()
print("Found Annotation with namespace: ", ann.getNs())

Found Annotation with namespace:  https://w3id.org/MontpellierRessourcesImagerie/microscopemetrics-schema/FieldIlluminationInput


In [40]:
ann = dataset_4.getAnnotation()
print("Found Annotation with namespace: ", ann.getNs())

Found Annotation with namespace:  https://w3id.org/MontpellierRessourcesImagerie/microscopemetrics-schema/FieldIlluminationInput


In [64]:
def get_dataset_ids_lists(conn, project_id):
    """
    Get the processed and unprocessed dataset ids for a project
    """
    processed_datasets = []
    unprocessed_datasets = []
    project = conn.getObject("Project", project_id)
    for dataset in project.listChildren():
        try:
            dataset.getAnnotation().getNs()
            processed_datasets.append(dataset.getId())
        except AttributeError:
            unprocessed_datasets.append(dataset.getId())
    return processed_datasets, unprocessed_datasets

In [65]:
l, n = get_dataset_ids_lists(conn, 3)

In [67]:
cxt = {'processed_datasets': l, 'unprocessed_datasets': n}

In [69]:
cxt["processed_datasets"]

[4, 5]

In [ ]:
def get_map_annotation(dataset_wrapper, conn):
    df = None
    for i in dataset_wrapper.listAnnotations():
        if i.__class__.__name__ == "MapAnnotationWrapper":
            keyPairs = conn.getObject("MapAnnotation", i.getId())
            table = dict(keyPairs.getValue())
            df = pd.DataFrame(table.items(), columns=["Key", "Value"])
    return df


In [34]:
def get_dataset_mapAnnotation(conn, dataset_id):
    """
    Get the mapAnnotation for a dataset
    """
    dataset = conn.getObject("Dataset", dataset_id)
    try:
        for i in dataset.listAnnotations():
            if "FieldIlluminationKeyValues" in i.getNs():
                table = dict(i.getValue())
                df = pd.DataFrame(table.items(), columns=["Key", "Value"])
                break
        return df
    except:
        return None

In [38]:
get_dataset_mapAnnotation(conn, 52)

,Key,Value
0,name,null
1,description,null
2,data_reference,null
3,linked_references,[]
4,channel_name,"[""DAPI"", ""FITC"", ""TRITC"", ""CY5""]"
5,center_region_intensity_fraction,"[0.05, 0.05, 0.05, 0.05]"
6,center_region_area_fraction,"[0.14889144897460938, 0.13515090942382812, 0.1..."
7,center_of_mass_y,"[294.9974891752709, 245.00832651217928, 206.99..."
8,center_of_mass_y_relative,"[0.152333942090902, -0.042936224561799685, -0...."
9,center_of_mass_x,"[297.0006917578335, 227.01518530017782, 301.00..."


In [41]:
for i in dataset_4.listAnnotations():
    print(i.getNs.getValue())

AttributeError: 'function' object has no attribute 'getValue'

In [16]:
import pandas as pd

In [96]:
a = np.arange(12).reshape(3, 4)
np.fliplr(a).diagonal()  # Horizontal flip


array([3, 6, 9])

In [65]:
import numpy as np

In [73]:
np.fliprl(a).diagonal()  # Vertical flip


AttributeError: module 'numpy' has no attribute 'fliprl'

In [97]:
b = np.flip(a, axis=1)
print(b[:,-1])
print(b[:,0])
print(np.fliplr(b).diagonal())
print(np.fliplr(a).diagonal())

[0 4 8]
[ 3  7 11]
[ 0  5 10]
[3 6 9]


In [79]:
np.fliplr(b).diagonal()

array([0, 4, 8])

In [87]:
b[:,-1]

array([0, 3, 6])

In [98]:
b

array([[ 3,  2,  1,  0],
       [ 7,  6,  5,  4],
       [11, 10,  9,  8]])

In [93]:
from .load import *

ImportError: attempted relative import with no known parent package